# Prétraitement des données

## Suppression des éléments indésirables de nos tweets

In [438]:
# Imports nécessaires
import pandas as pd
import numpy as np
import string 
import re
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

In [55]:
# Lecture et stockage de la base de données
tweet_df = pd.read_csv('../../delphes/data/final2_clean.csv', index_col=0)
tweet_df.head()

,mep_id,name,country,group,nat_group,twitter,content
0,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,"['W tym dniu, w tym miejscu, w tej godzinie pr..."
1,189525,Asim ADEMOV,Bulgaria,Group of the European People's Party (Christia...,Citizens for European Development of Bulgaria,AdemovAsim,['RT @ECinBulgaria: 📢 Остана 1⃣ седмица! Преди...
2,124831,Isabella ADINOLFI,Italy,Non-attached Members,Movimento 5 Stelle,Isa_Adinolfi,"[""Sembra un film, ma purtroppo è realtà: le im..."
6,125045,Clara AGUILERA,Spain,Group of the Progressive Alliance of Socialist...,Partido Socialista Obrero Español,ClaraAguilera7,['RT @ClaraAguilera7: Debate e importantes vot...
7,204335,Alviina ALAMETSÄ,Finland,Group of the Greens/European Free Alliance,Vihreä liitto,alviinaalametsa,['Toimeentulotukea korotetaan 75e koronakriisi...


In [56]:
# Remove the undesirable elements in the entire dataframe
def rmurl_df(df, column_name):
    '''
    This function removes all the URLs, the #hashtag and the @user of a column made of strings.
    Be careful to apply it BEFORE all the other preprocessing steps (if not it wont'
    be recognized as a URL)
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.replace('http\S+|www.\S+|@\S+|#\S+', '', case=False)
    return df



In [57]:
# Lowercase the tweet's column
def lower_df(df, column_name):
    '''
    This function lowercases a column made of strings.
    '''
    df = df.copy()
    df[column_name] = df[column_name].str.lower()
    return df

In [58]:
# Remove the numbers in the tweet's column
def rmnumbers_df(df, column_name):
    '''
    This function removes all the digits of a column made of strings.
    '''
    df = df.copy()
    def remove_numbers(text):
        return ''.join(word for word in text if not word.isdigit())
    df[column_name] = df[column_name].apply(remove_numbers)
    return df

In [517]:
# Remove the undesirable punctuations in the tweet's column
def rmpunct_df(df, column_name):
    '''
    This function removes all the punctuations, all the "rt" and remove multiple spaces
    of a column made of strings.
    '''
    punct = string.punctuation
    df = df.copy()
    def replace_punct(text):
        for punctu in punct:
            text = text.replace(punctu, ' ')
        text = text.replace(' rt ','')
        text = " ".join(text.split())
        return text
    df[column_name] = df[column_name].apply(replace_punct)
    return df

In [573]:
# Remove the stopwords in the tweet's column
def rmstopwords_df(df, column_name):
    '''
    This function removes all the stopwords of a column made of strings.
    '''
    df = df.copy()
    stop_words = stopwords.words('english')
    def remove_stopwords(text):
        for word in stop_words:
            text = text.replace(f' {word} ', ' ')
        return text
    df[column_name] = df[column_name].apply(remove_stopwords)
    return df

In [574]:
# Remove the undesirable emojis in the entire dataframe
def rmemojis_df(df):
    '''
    This function removes all the emojis of a column made of strings.
    Be careful to translate in latin alphabet before applying this function : 
    it also removes cyrillic alphabet
    '''
    df = df.copy()
    df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))
    return df

In [577]:
clean_df = rmurl_df(tweet_df, 'content')
clean_df = lower_df(clean_df, 'content')
clean_df = rmnumbers_df(clean_df, 'content')
clean_df = rmpunct_df(clean_df, 'content')
clean_df = rmstopwords_df(clean_df, 'content')
clean_df = rmemojis_df(clean_df)

In [578]:
clean_df[clean_df['country'] == 'Ireland'].head()

,mep_id,name,country,group,nat_group,twitter,content
136,124988,Deirdre CLUNE,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,deirdreclunemep,the guidance preparing end transition period s...
146,197654,Ciarn CUFFE,Ireland,Group of the Greens/European Free Alliance,Green Party,ciarancuffe,it crucial follows phil hogan replacement big ...
208,197720,Frances FITZGERALD,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,FitzgeraldFrncs,leaving london talks w continue difficult neg...
209,124985,Luke Ming FLANAGAN,Ireland,Group of the European United Left - Nordic Gre...,Independent,lukeming,the meat sector truly give toss n nit would co...
335,96668,Sen KELLY,Ireland,Group of the European People's Party (Christia...,Fine Gael Party,SeanKellyMEP,eu member sates reinstate solvency support ins...


In [579]:
testbase_df = clean_df[clean_df['country'] == 'Ireland']

## Premier test Word2Vec

In [580]:
testbase_df = testbase_df[['mep_id', 'content']]

In [581]:
from gensim.models import Word2Vec

In [582]:
sentences = testbase_df['content']

In [602]:
sentences_train = []
for sentence in sentences:
    sentences_train.append(sentence.split())

In [603]:
word2vec = Word2Vec(sentences=sentences_train)

In [604]:
def embed_sentence(word2vec, sentence):
    y = []
    for word in sentence:
        if word in word2vec.wv.vocab.keys():
           y.append(word2vec[word])
    return np.array(y)

def embedding(word2vec, sentences):
    
    y = []
    for sentence in sentences:
        y.append(embed_sentence(word2vec, sentence))
    return y

In [605]:
X_train = embedding(word2vec,sentences_train)

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [606]:
from tensorflow.keras.utils import to_categorical
y_train = np.arange(0,10,1)
y_cat_train = to_categorical(y_train, num_classes=10)

In [607]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

def init_model():

    model = Sequential()
    model.add(layers.Masking(mask_value = -1000))
    model.add(layers.LSTM(13))
    model.add(layers.Dense(10, activation='relu'))
    model.add(layers.Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop', 
              metrics=['accuracy'])
    return model

model = init_model()

In [608]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train_pad = pad_sequences(X_train, padding='post',value=-1000, dtype='float32')

In [609]:
X_train_pad.shape

(10, 2319, 100)

In [610]:
y_cat_train.shape

(10, 10)

In [613]:
model = init_model()
model.fit(X_train_pad, y_cat_train, epochs=100)

Train on 10 samples
Epoch 1/100
10/10 [==============================] - 2s 216ms/sample - loss: 2.3016 - acc: 0.1000
Epoch 2/100
10/10 [==============================] - 1s 135ms/sample - loss: 2.2957 - acc: 0.1000
Epoch 3/100
10/10 [==============================] - 1s 135ms/sample - loss: 2.2932 - acc: 0.1000
Epoch 4/100
10/10 [==============================] - 1s 135ms/sample - loss: 2.2906 - acc: 0.1000
Epoch 5/100
10/10 [==============================] - 1s 136ms/sample - loss: 2.2892 - acc: 0.1000
Epoch 6/100
10/10 [==============================] - 1s 134ms/sample - loss: 2.2879 - acc: 0.1000
Epoch 7/100
10/10 [==============================] - 1s 133ms/sample - loss: 2.2844 - acc: 0.1000
Epoch 8/100
10/10 [==============================] - 1s 135ms/sample - loss: 2.2826 - acc: 0.1000
Epoch 9/100
10/10 [==============================] - 1s 136ms/sample - loss: 2.2804 - acc: 0.1000
Epoch 10/100
10/10 [==============================] - 1s 136ms/sample - loss: 2.2778 - acc: 0.1000

Epoch 84/100
10/10 [==============================] - 1s 149ms/sample - loss: 1.7521 - acc: 0.5000
Epoch 85/100
10/10 [==============================] - 1s 134ms/sample - loss: 1.7452 - acc: 0.4000
Epoch 86/100
10/10 [==============================] - 2s 152ms/sample - loss: 1.7306 - acc: 0.5000
Epoch 87/100
10/10 [==============================] - 1s 147ms/sample - loss: 1.7241 - acc: 0.4000
Epoch 88/100
10/10 [==============================] - 1s 133ms/sample - loss: 1.7411 - acc: 0.3000
Epoch 89/100
10/10 [==============================] - 1s 135ms/sample - loss: 1.7121 - acc: 0.5000
Epoch 90/100
10/10 [==============================] - 1s 132ms/sample - loss: 1.7245 - acc: 0.6000
Epoch 91/100
10/10 [==============================] - 1s 140ms/sample - loss: 1.7279 - acc: 0.4000
Epoch 92/100
10/10 [==============================] - 1s 140ms/sample - loss: 1.6909 - acc: 0.4000
Epoch 93/100
10/10 [==============================] - 1s 141ms/sample - loss: 1.6884 - acc: 0.5000
Epoch 94/1

In [620]:
examples2 = ['morning Deputy Dara Calleary tendered resignation Minister Agriculture Food Marine following attendance Oireachtas golf dinner Wednesday evening attendance event wrong error judgement part accepted resignation People country have made difficult personal sacrifices family lives businesses comply COVID-19 regulations event should gone ahead manner given Government decision last Tuesday Dara Calleary since first elected Dail Eireann remains committed dedicated public representative error judgement character made right decision country particularly light continued efforts suppress COVID-19',
            'guidance anger detect deputy']

In [621]:
examples2[0] = examples2[0].lower()

In [622]:
examples=[]
for example in examples2:
    examples.append(example.split())

In [623]:
examples[1]

['guidance', 'anger', 'detect', 'deputy']

In [624]:
X_test = embedding(word2vec, examples)
X_test_pad = pad_sequences(X_test, padding='post',value=-1000, dtype='float32')

c:\users\psmag\.venvs\delphes\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [625]:
model.predict(X_test_pad)

array([[0.10833959, 0.08891478, 0.07604273, 0.12035228, 0.10818799,
        0.09217536, 0.08763052, 0.10064937, 0.10162988, 0.11607757],
       [0.10745704, 0.10042856, 0.10160366, 0.08884913, 0.09296093,
        0.1151541 , 0.08331128, 0.11196935, 0.11317366, 0.08509227]],
      dtype=float32)